In [210]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import math
import time 
import random

In [211]:
url = 'https://movie.douban.com/subject/34779776/comments?start=0&limit=40&status=P&sort=new_score'
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
response = requests.get(url=url, headers=headers)

In [212]:
response

<Response [200]>

In [213]:
html = response.text
# parse html
soup = BeautifulSoup(html, 'html.parser')

In [214]:
soup

<script>var d=[navigator.platform,navigator.userAgent,navigator.vendor].join("|");window.location.href="https://sec.douban.com/a?c=184a5a&d="+d+"&r=https%3A%2F%2Fmovie.douban.com%2Fsubject%2F34779776%2Fcomments%3Fstart%3D0%26limit%3D40%26status%3DP%26sort%3Dnew_score&k=CDplwlurVrDIkP42bOnX3VhL%2ByAkOzKRLcjkEnCjCMQ";</script>

In [190]:
# total number of comments
item_num = soup.find(class_='is-active').find('span').text
item_num = int(re.findall(r'\d+', item_num)[0])
item_num

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
# comments in the first page
items = soup.find_all('div', attrs={'class': 'comment-item'})
len(items)

In [ ]:
# the first comment item on the first page
first = items[0]
first

In [ ]:
# username
account_title = first.find(class_='avatar').find('a').get('title')
# link to user profile
account_link = first.find(class_='avatar').find('a').get('href')
# number of votes
vote_num = first.find(class_='votes vote-count').text
vote_num = int(vote_num)
# rating
star = first.find(class_='rating').get('title')
# comment time
comment_time = first.find(class_='comment-time').get('title')
# comment location, some do not exist
comment_location = first.find(class_='comment-location').text
# comment string
comment = first.find(class_='comment-content').text.strip()
comment

In [ ]:
# collect data on the first page
data_dic_list = []
for i in items:
    account_title = i.find(class_='avatar').find('a').get('title')
    account_link = i.find(class_='avatar').find('a').get('href')
    vote_num = i.find(class_='votes vote-count').text
    vote_num = int(vote_num)
    try:
        star = i.find(class_='rating').get('title') 
    except:
        star = None
    comment_time = i.find(class_='comment-time').get('title')
    comment_location = i.find(class_='comment-location').text
    comment = i.find(class_='comment-content').text.strip()
    dic = {
        'username': account_title,
        'profile_link': account_link,
        'vote_num': vote_num,
        'star': star,
        'comment_time': comment_time,
        'comment_location': comment_location,
        'comment': comment
    }
    data_dic_list.append(dic)

In [ ]:
pd.DataFrame(data_dic_list)

In [ ]:
def get_items(page):
    url = f'https://movie.douban.com/subject/34779776/comments?start={page}&limit=100&status=P&sort=new_score'
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36",
    }
    response = requests.get(url=url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', attrs={'class': 'comment-item'})
    return items

In [ ]:
def get_comments(data_dic_list, items):
    for i in items:
        account_title = i.find(class_='avatar').find('a').get('title')
        account_link = i.find(class_='avatar').find('a').get('href')
        vote_num = i.find(class_='votes vote-count').text
        vote_num = int(vote_num)
        try:
            star = i.find(class_='rating').get('title') 
        except:
            star = None
        comment_time = i.find(class_='comment-time').get('title')
        comment_location = i.find(class_='comment-location').text
        comment = i.find(class_='comment-content').text.strip()
        dic = {
            'account_title': account_title,
            'account_link': account_link,
            'vote_num': vote_num,
            'star': star,
            'comment_time': comment_time,
            'comment_location': comment_location,
            'comment': comment
        }
        data_dic_list.append(dic)

In [ ]:
# total pages
total_pages = math.ceil(item_num / 100)
total_pages 

In [ ]:
data_dic_list = []
for page in range(0, 3):
    items = get_items(page)
    get_comments(data_dic_list, items)
    time.sleep(0.1) 
    print(f'{page} is done')

In [ ]:
df = pd.DataFrame(data_dic_list)
# df.to_csv('canglangjue_data.csv', index=False)
df